In [17]:
import pandas as pd
import torch

dev_data = pd.read_csv('hotel/dev.csv')
train_data = pd.read_csv('hotel/train.csv')
test_data = pd.read_csv('hotel/test.csv')

In [18]:
from transformers import DistilBertTokenizer

# 初始化tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [19]:
from torch.utils.data import Dataset
class ReviewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx] - 1, dtype=torch.long)  # 确保标签是长整型
        return item

    def __len__(self):
        return len(self.labels)
train_encodings = tokenizer(list(train_data['reviews.text']), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_data['reviews.text']), truncation=True, padding=True, max_length=128)

train_labels = list(train_data['reviews.rating'])
test_labels = list(test_data['reviews.rating'])

train_dataset = ReviewsDataset(train_encodings.data, train_labels)  # 使用 .data 属性来访问encodings
test_dataset = ReviewsDataset(test_encodings.data, test_labels)




In [20]:
from transformers import DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=5)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

c:\Users\jrzcn\AppData\Local\Programs\Python\Python311\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [22]:
trainer.train()

  0%|          | 0/6339 [00:00<?, ?it/s]

IndexError: Target 7 is out of bounds.